# Elevation Data Ingest Tutorial

In this project, elevation above sea level is utilized as a predictor of FMC. RAWS stations report a elevation that is used when constructing training data.

For forecasting to arbitrary locations, we need elevation data. The HRRR weather model utilizes a geopotential height in it's modeling, but this may not be directly comparable to the elevation as reported by a RAWS station. In this project, we will utilize elevation data from [LandFire](https://landfire.gov/topographic/elevation), which utilizes 3D mapping from USGS. The elevation data is very high resolution; it has approximately 30m resolution and is contained in a file that is over 8gb. 

In this project we utilize the LandFire elevation data as follows:
* Download the LandFire data for all of CONUS (this takes while)
* Read in one HRRR grib file to get grid nodes and projection info
* Reproject the HRRR file to the LandFire projection
* Interpolate the LandFire elevation data to the location of the reprojected HRRR nodes
* Save a local copy of this projected info and stage on OpenWFM Demo. This data corresponds to the LandFire elevation data interpolated to the HRRR grid nodes. This elevation will be used as a predictor for forecasting FMC at a given HRRR grid node

This notebook demonstrates the process of downloading the LandFire elevation data and extracting points that correspond to the HRRR grid. This is a slow process that should only be run on project setup once. Additionally, the elevation data at HRRR grid nodes can be downloaded online, thus avoiding this time consuming process, but the code is included for reproducibility.

*NOTE*: LandFire data too big for processing in memory in a jupyter notebook. TODO: use gdal command line. Reference: https://nsidc.org/data/user-resources/help-center/how-do-i-reproject-geotiff-polar-stereographic-geographic-latlon

## Setup

In [ ]:
import zipfile
import sys
import os.path as osp
import rioxarray as rxr
sys.path.append("../src")
from utils import retrieve_url

In [ ]:
landfire_url = "https://landfire.gov/data-downloads/US_Topo_2020/LF2020_Elev_220_CONUS.zip"
data_path = "../data"
zipname = "LF2020_Elev_220_CONUS.zip"
elev_path = osp.join(data_path, "LF2020_Elev_220_CONUS", "Tif", "LC20_Elev_220.tif")

## Retrieve and Read

In [ ]:
retrieve_url(
    url = landfire_url,
    dest_path = osp.join(data_path, zipname)
)

if not osp.exists(elev_path):
    print("Unzipping file")
    with zipfile.ZipFile(osp.join(data_path, filename), 'r') as zip_ref:
        zip_ref.extractall(data_path)    

In [ ]:
ds = rxr.open_rasterio(elev_path)

In [ ]:
ds

In [ ]:
ds.rio.crs

In [ ]:
import herbie

In [ ]:
ds2 = herbie.Herbie("2024-12-01", product="prs").xarray("(?:HGT|LAND):surface")

In [ ]:
ds2.herbie.crs

In [ ]:
ds2

In [ ]:
ds2.orog.max()

In [ ]:
ds2.dims

In [ ]:
ds2.coords

In [ ]:
## DOESNT Work below, from xarray example and ChatGPT
# ds2 = ds2.rename({"longitude": "x", "latitude": "y"})
# ds2_reprojected = ds2.rio.reproject(ds.rio.crs)
# ds2_reprojected = ds2.rio.reproject("EPSG:5070")

In [ ]:
H = herbie.Herbie("2024-12-01", product="prs")

In [ ]:
path = H.download("(?:HGT|LAND):surface")

In [ ]:
print(path)

In [ ]:
ds3 = rxr.open_rasterio(path, engine="cfgrib")

In [ ]:
ds3

In [ ]:
ds3_reprojected = ds3.rio.reproject(ds.rio.crs)

In [ ]:
ds3

In [ ]:
ds3_reprojected

In [ ]:
## KILLS the kernel
# ds3_reprojected['interpolated_values'] = ds.interp(
#     x=ds3_reprojected['x'],
#     y=ds3_reprojected['y'],
#     method="nearest" 
# )